# Natural Map

## imports

In [1]:
import JordanAlgebra
from sympy import Symbol, Matrix, sqrt, Pow, Add, Rational, postorder_traversal, factor, simplify, nsimplify, latex
from codegen.funcodegen import funcodegen
import numpy as np


## symbols

In [2]:
from JordanAlgebra import epsilon
from friction import mu, rn, rt1, rt2, un, ut1, ut2, xn, xt1, xt2, yn, yt1, yt2, u, r, x, y, ut


## The natural map function

In [3]:
Fnat = y - JordanAlgebra.projection(y - x)


## Raw jacobians

In [4]:
A_ = Fnat.jacobian(u)
B_ = Fnat.jacobian(r)

In [5]:
lambda1, lambda2, u1, u2 = JordanAlgebra.spectral_decomposition(y - x)

#def load(v):

#    with open(v) as r_file:
#        return pickle.load(r_file)

#A = load('Fnat_A')
#B = load('Fnat_B')
#Fnat = load('Fnat')
#Rnow = load('Fnat_Rnow')

In [63]:
from sympy import Piecewise
def add_split(expr, pos, f1, f2):
    n_expr = expr
    if type(expr) == Add:       
       n_expr1 = Add(*expr.args[:pos])
       n_expr2 = Add(*expr.args[pos:])
       n_expr = Add(f2(n_expr2), f1(n_expr1))
    return n_expr

def add_splitp(p_expr, lpos, f1, f2):
    if hasattr(p_expr, 'is_Piecewise') and p_expr.is_Piecewise:
        return Piecewise(*[(add_split(c[0], lpos[i], f1, f2), c[1]) for i, c in enumerate(p_expr.args)])
    else:
        return add_split(p_expr, lpos[0], f1, f2)

def add_splitv(mat, lpos, f1, f2):
    return Matrix(mat.shape[0], mat.shape[1], 
        lambda i,j: add_splitp(mat[i, j], lpos[i], f1, f2))

def map_matrix(mat, fun, args):
    return Matrix(mat.shape[0], mat.shape[1], 
                  lambda i,j: fun(mat[i, j], *(args[i])))

from sympy import Mul, Add, Pow, Wild
def factor_try(input_expr, fexprs):
    n_expr = input_expr
    subs = dict()   
    for fexpr in fexprs:
        for expr in postorder_traversal(n_expr):
            niexpr = expr   
            for iexpr in subs:
               niexpr = niexpr.subs(iexpr, subs[iexpr])
            expr = niexpr
            if type(expr) == Add:
               fterms = filter(lambda e: type(e)==Mul and fexpr in e.args, expr.args)
               uterms = filter(lambda e: type(e)==Mul and any([fexpr == ee.args[0] for ee in filter(lambda e: type(e)==Pow, e.args)]), expr.args)              
               pterms = filter(lambda e: type(e)==Pow and e.args[1].is_Integer and fexpr == e.args[0], expr.args)
               tterms = filter(lambda e: e == fexpr, expr.args)
               oterms = list(set(expr.args).difference(set(uterms)).difference(set(pterms)).difference(set(tterms)).difference(set(fterms)))
               nfterms = list()
               for e in fterms:
                   nargs = list(e.args)
                   nargs.remove(fexpr)
                   nfterms.append(e.func(*nargs))
               for e in uterms:
                   ne = 1
                   wuterms = filter(lambda ie: type(ie)==Pow and ie.args[0]==fexpr, e.args)
                   for ee in wuterms:
                       ne = Mul(ne, Pow(ee.args[0], ee.args[1]-1))
                   for ee in set(e.args).difference(wuterms):
                       ne = Mul(ne, ee)
                   nfterms.append(ne)
               for e in pterms:
                   nfterms.append(e.func(e.args[0], e.args[1]-1))
               for e in tterms:
                   nfterms.append(1)
               
               nfresult = fexpr * Add(*nfterms)
              
               n_expr = n_expr.subs(expr, Add(Add(*oterms), nfresult))
               subs[expr] = Add(Add(*oterms), nfresult)
    return n_expr
       


In [7]:
from sympy import Wild
expr = sqrt(rt1)
y = Wild('y')
expr.replace(Pow(y, Rational(3,2)), lambda y: y+1)

sqrt(rt1)

In [65]:
factor_try(sqrt(2)*ut1*rt1**4+rt1*ut1+sqrt(2)*rt1*ut2, [sqrt(2), rt1])

    ⎛        ___ ⎛   3          ⎞⎞
rt₁⋅⎝ut₁ + ╲╱ 2 ⋅⎝rt₁ ⋅ut₁ + ut₂⎠⎠

In [66]:
factor_try(sqrt(rt1+ut1*rt1**7)+ ut1*rt1 + ut2*rt1*rt1, [rt1,2])

                         ____________________
                        ╱     ⎛   6        ⎞ 
rt₁⋅(rt₁⋅ut₂ + ut₁) + ╲╱  rt₁⋅⎝rt₁ ⋅ut₁ + 1⎠ 

In [10]:
add_splitp(Piecewise((1 + ut1**2/2 + ut2**2/2, ut1>0), (ut1, ut1<=0)), [1,0], factor, factor)

#map_matrix(Matrix([1 + ut1**2/2 + ut2**2/2]), add_split, [1, factor, factor])

Piecewise(((ut1**2 + ut2**2)/2 + 1, ut1 > 0), (ut1, ut1 <= 0))

ndelta, sqrt((mu*ut2-rt2)**2+(mu*ut1-rt1)**2)

In [11]:
from sympy import init_printing
init_printing(use_latex=True)

from sympy import Symbol
nut = Symbol(r'\lVert {u_{t}} \rVert', negative=False)
nrt = Symbol(r'\lVert {r_{t}} \rVert', negative=False)
lbd1 = Symbol(r'\lambda_1')
lbd2 = Symbol(r'\lambda_2')
lbd10 = Symbol(r'\lambda_1^{\prime}')
lbd20 = Symbol(r'\lambda_2^{\prime}')
lbd100 = Symbol(r'\lambda_1^{\prime\prime}')
lbd200 = Symbol(r'\lambda_2^{\prime\prime}')
delta1 = Symbol(r'\Delta_1')
delta2 = Symbol(r'\Delta_2')
ndelta = Symbol(r'\lVert {\Delta} \rVert', negative=False)
lepsilon = Symbol(r'\epsilon')
lut1 = Symbol(r'u_{t1}')
lut2 = Symbol(r'u_{t2}')
lun = Symbol(r'u_n')
lrt1 = Symbol(r'r_{t1}')
lrt2 = Symbol(r'r_{t2}')
lrn = Symbol(r'r_n')

from codegen.funcodegen import flatten_piecewise

lambda1_0 = lambda1.subs(ut1, 0).subs(ut2, 0)
lambda2_0 = lambda2.subs(ut1, 0).subs(ut2, 0)

lambda1_00 = lambda1_0.subs(rt1, 0).subs(rt2, 0)
lambda2_00 = lambda2_0.subs(rt1, 0).subs(rt2, 0)

def make_readable(expr, deep=False):
    return flatten_piecewise(expr).subs(lrt1, rt1).subs(lrt2, rt2).subs(lrn, rn).subs(lut1, ut1).subs(lut2, ut2).subs(lun, un).subs(nut, sqrt(ut1**2+ut2**2)).subs(nrt, sqrt(rt1**2+rt2**2)).subs('rt1', rt1).subs('rt2', rt2).subs('rn', rn).subs('ut1', ut1).subs('ut2', ut2).subs('un', un).subs('mu', mu).subs(lambda1, lbd1).subs(lambda2, lbd2).subs(lambda1_0, lbd10).subs(lambda2_0, lbd20).subs(lambda1_00, lbd100).subs(lambda2_00, lbd200).subs(ut1**2+ut2**2,nut**2).subs(rt1**2+rt2**2, nrt**2).subs(mu*ut2-rt2, delta2).subs(mu*ut1-rt1, delta1).subs(delta1**2+delta2**2, ndelta**2).subs(rt1, lrt1).subs(rt2, lrt2).subs(rn, lrn).subs(ut1, lut1).subs(ut2, lut2).subs(un, lun).subs(epsilon, lepsilon).subs(0.5, Rational(1, 2)).subs(sqrt(nrt**2), nrt).subs(sqrt(nut**2), nut).subs(sqrt(ndelta**2), ndelta).subs(sqrt(sqrt(nrt**2)), sqrt(nrt))


In [12]:
from sympy import expand
add_splitv(make_readable(Fnat), [[2], [1,0], [1,1]], factor, factor)


⎡                                  Max(0, \lambda_1) + Max(0, \lambda_2)      
⎢                          μ⋅r_n - ─────────────────────────────────────      
⎢                                                    2                        
⎢                                                                             
⎢⎧  \Delta₁⋅(Max(0, \lambda_1) - Max(0, \lambda_2))                           
⎢⎪- ─────────────────────────────────────────────── + r_{t1}  for \lVert {\Del
⎢⎨              2⋅\lVert {\Delta} \rVert                                      
⎢⎪                                                                            
⎢⎩                          r_{t1}                            for \lVert {\Del
⎢                                                                             
⎢⎧  \Delta₂⋅(Max(0, \lambda_1) - Max(0, \lambda_2))                           
⎢⎪- ─────────────────────────────────────────────── + r_{t2}  for \lVert {\Del
⎢⎪              2⋅\lVert {\Delta} \rVert            

In [13]:
P=make_readable(Fnat[1]).args[0][0]



In [14]:
add_split(make_readable(Fnat[2]), 2, factor, factor)

⎧  \Delta₂⋅Max(0, \lambda_1)   \Delta₂⋅Max(0, \lambda_2)                      
⎪- ───────────────────────── + ───────────────────────── + r_{t2}  for \lVert 
⎪   2⋅\lVert {\Delta} \rVert    2⋅\lVert {\Delta} \rVert                      
⎨                                                                             
⎪                  Max(0, \lambda_1)   Max(0, \lambda_2)                      
⎪         r_{t2} + ───────────────── - ─────────────────           for \lVert 
⎩                          2                   2                              

                          
{\Delta} \rVert > \epsilon
                          
                          
                          
{\Delta} \rVert ≤ \epsilon
                          

In [15]:
with open('codegen/tex/fnat.tex', 'w') as f:
    f.write(latex(add_splitv(make_readable(Fnat), [[2], [1,0], [1,1]], factor, factor)))

In [16]:
from codegen.maple import limzero, Maple, set_maple
maple = Maple(server='bizet.inria.fr')
set_maple(maple)
t = Symbol('t', positive=True, real=True)
maple_assumes = 'mu >= 0 and mu < 1 and rn >=0 and t > 0 and epsilon > 0'
maple.assume(maple_assumes)


No remote temporary directory (option server_tmpdir) specified, using /tmp/ on bizet.inria.fr


In [17]:
def nsimp(expr):
    n_expr = expr
    for subexpr in postorder_traversal(expr):
        n_expr = n_expr.subs(subexpr, nsimplify(subexpr))
    return n_expr

In [18]:
A=Matrix(3, 3, lambda i, j: 0)

## A[:, 0]

In [19]:
add_splitv(make_readable(A_[:, 0]), [[2], [2,0], [2,0]], factor, factor)

⎡                         Heaviside(\lambda₁) + Heaviside(\lambda₂)           
⎢                         ─────────────────────────────────────────           
⎢                                             2                               
⎢                                                                             
⎢⎧\Delta₁⋅(Heaviside(\lambda₁) - Heaviside(\lambda₂))                         
⎢⎪───────────────────────────────────────────────────  for \lVert {\Delta} \rV
⎢⎨              2⋅\lVert {\Delta} \rVert                                      
⎢⎪                                                                            
⎢⎩                         0                           for \lVert {\Delta} \rV
⎢                                                                             
⎢⎧\Delta₂⋅(Heaviside(\lambda₁) - Heaviside(\lambda₂))                         
⎢⎪───────────────────────────────────────────────────  for \lVert {\Delta} \rV
⎢⎪              2⋅\lVert {\Delta} \rVert            

In [20]:
# the first column is ok
A[:, 0] = A_[:, 0]

In [21]:
with open('codegen/tex/A0_fnat.tex', 'w') as f:
     f.write(latex(add_splitv(make_readable(A[:, 0]), [[2], [2,0], [2,0]], factor, factor)))

## A[0, 1]

In [22]:
factor(make_readable(A_[0, 1]))

μ⋅(\Delta₁⋅\lVert {u_{t}} \rVert⋅Heaviside(\lambda₁) - \Delta₁⋅\lVert {u_{t}} 
──────────────────────────────────────────────────────────────────────────────
                                                                              

\rVert⋅Heaviside(\lambda₂) + \lVert {\Delta} \rVert⋅u_{t1}⋅Heaviside(\lambda₁)
──────────────────────────────────────────────────────────────────────────────
    2⋅\lVert {\Delta} \rVert⋅\lVert {u_{t}} \rVert                            

 + \lVert {\Delta} \rVert⋅u_{t1}⋅Heaviside(\lambda₂))
─────────────────────────────────────────────────────
                                                     

In [23]:
# we must apply the same limits or subsitution for all the jacobian
# unfortunately substituing 0 for both ut1 and ut2 (in the same order) leads to some NaN
make_readable(nsimp(limzero(A_[0, 1], [(ut1, t), (ut2, t)])))

  ⎛  ___                                                          ___         
μ⋅⎝╲╱ 2 ⋅\lVert {r_{t}} \rVert⋅Heaviside(\lambda_1__{\prime}) + ╲╱ 2 ⋅\lVert {
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
r_{t}} \rVert⋅Heaviside(\lambda_2__{\prime}) - 2⋅r_{t1}⋅Heaviside(\lambda_1__{
──────────────────────────────────────────────────────────────────────────────
              4⋅\lVert {r_{t}} \rVert                                         

                                                  ⎞
\prime}) + 2⋅r_{t1}⋅Heaviside(\lambda_2__{\prime})⎠
───────────────────────────────────────────────────
                                                   

pour \Delta_n différent de 0, ||u_t|| -> 0 (==> rt != 0)

deltan -> 0

In [24]:
from codegen.maple import mlimit
make_readable((nsimp(mlimit(A_[0, 1].subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0))))

μ⋅r_{t1}⋅Heaviside(\lambda_1__{\prime})
───────────────────────────────────────
         \lVert {r_{t}} \rVert         

deltan -> 0 && ut -> 0 ==> rt -> 0

In [146]:
make_readable(nsimp(limzero(A_[0, 1], [(ut1, t), (ut2, t), (rt1, t), (rt2, 0)])))


  ___                                       
╲╱ 2 ⋅μ⋅Heaviside(\lambda_1__{\prime\prime})
────────────────────────────────────────────
                     2                      

In [165]:
def natfix(expr):
    return Piecewise(
     (simplify(nsimp(limzero(expr, [(ut1, t), (ut2, t), (rt1, t), (rt2, 0)]))), And(sqrt(ut1**2+ut2**2) <= epsilon, sqrt(rt1**2+rt2**2) <= epsilon)),
     (simplify(nsimp(mlimit(expr.subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0))), sqrt((mu*ut1-rt1)**2+(mu*ut2-rt2)**2) <= epsilon),
     (expr, And(sqrt(ut1**2+ut2**2) > epsilon, sqrt((mu*ut1-rt1)**2+(mu*ut2-rt2)**2) > epsilon)),
     (simplify(nsimp(limzero(expr, [(ut1, t), (ut2, t)]))), And(sqrt((mu*ut1-rt1)**2+(mu*ut2-rt2)**2) > epsilon, sqrt(ut1**2+ut2**2) <= epsilon, sqrt(rt1**2+rt2**2) > epsilon)))

In [166]:
from sympy import And
A[0, 1] = natfix(A_[0, 1])
     
nsimp(make_readable(A[0, 1]))

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                 ⎛        \Delta₁⋅μ                 μ⋅u_{t1} 
⎨                                 ⎜- ────────────────────── - ────────────────
⎪                                 ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \
⎪                               - ──────────────────

In [154]:
with open('codegen/tex/A01_fnat.tex', 'w') as f:
     f.write(latex(nsimp(make_readable(A[0, 1]))))

## A[0, 2]

In [28]:
make_readable(A_[0,2])

  ⎛        \Delta₂⋅μ                 μ⋅u_{t2}      ⎞                       ⎛  
  ⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambda₁)   ⎜──
  ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                       ⎝\l
- ────────────────────────────────────────────────────────────────────── - ───
                                    2                                         

    \Delta₂⋅μ                 μ⋅u_{t2}      ⎞                    
──────────────────── - ─────────────────────⎟⋅Heaviside(\lambda₂)
Vert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                    
─────────────────────────────────────────────────────────────────
                              2                                  

deltan != 0, ut -> 0

In [29]:
make_readable(nsimp(sqrt(lambda1_0)))

  _____________________
╲╱ \lambda_1__{\prime} 

In [30]:
make_readable(nsimp(limzero(A_[0, 2], [(ut1, t), (ut2, t)])))

  ⎛  ___                                                          ___         
μ⋅⎝╲╱ 2 ⋅\lVert {r_{t}} \rVert⋅Heaviside(\lambda_1__{\prime}) + ╲╱ 2 ⋅\lVert {
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
r_{t}} \rVert⋅Heaviside(\lambda_2__{\prime}) - 2⋅r_{t2}⋅Heaviside(\lambda_1__{
──────────────────────────────────────────────────────────────────────────────
              4⋅\lVert {r_{t}} \rVert                                         

                                                  ⎞
\prime}) + 2⋅r_{t2}⋅Heaviside(\lambda_2__{\prime})⎠
───────────────────────────────────────────────────
                                                   

In [147]:
make_readable(nsimp(limzero(A_[0, 2], [(ut1, t), (ut2, t), (rt1, t), (rt2, 0)])))

  ___                                       
╲╱ 2 ⋅μ⋅Heaviside(\lambda_1__{\prime\prime})
────────────────────────────────────────────
                     2                      

In [32]:
make_readable(nsimp(mlimit(A_[0, 2].subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0)))

μ⋅r_{t2}⋅Heaviside(\lambda_1__{\prime})
───────────────────────────────────────
         \lVert {r_{t}} \rVert         

In [167]:
A[0, 2] = natfix(A_[0, 2])
     
nsimp(make_readable(A[0, 2]))

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                 ⎛        \Delta₂⋅μ                 μ⋅u_{t2} 
⎨                                 ⎜- ────────────────────── - ────────────────
⎪                                 ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \
⎪                               - ──────────────────

In [156]:
with open('codegen/tex/A02_fnat.tex', 'w') as f:
     f.write(latex(nsimp(make_readable(A[0, 2]))))

## A[1, 0]

In [35]:
make_readable(A_[1, 0])

⎧\Delta₁⋅Heaviside(\lambda₁)   \Delta₁⋅Heaviside(\lambda₂)                    
⎪─────────────────────────── - ───────────────────────────  for \lVert {\Delta
⎨  2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert                     
⎪                                                                             
⎩                            0                              for \lVert {\Delta

                   
} \rVert > \epsilon
                   
                   
} \rVert ≤ \epsilon

In [36]:
# just a verification
mlimit(A_[1, 0].subs(ut1, (1+t)*rt1/mu).subs(ut2, (1+t)*rt2/mu), t, 0)

0.0

In [37]:
A[1, 0] = A_[1, 0]

In [38]:
factor(make_readable(A[1, 0]))

⎧\Delta₁⋅Heaviside(\lambda₁)   \Delta₁⋅Heaviside(\lambda₂)                    
⎪─────────────────────────── - ───────────────────────────  for \lVert {\Delta
⎨  2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert                     
⎪                                                                             
⎩                            0                              for \lVert {\Delta

                   
} \rVert > \epsilon
                   
                   
} \rVert ≤ \epsilon

In [39]:
with open('codegen/tex/A10_fnat.tex', 'w') as f:
     f.write(latex(factor(make_readable(A[1, 0]))))

## A[1, 1]

In [157]:
make_readable(A_[1, 1])

⎧                                                                             
⎪                                                                             
⎪          ⎛        \Delta₁⋅μ                 μ⋅u_{t1}      ⎞                 
⎪  \Delta₁⋅⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambd
⎨          ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                 
⎪- ───────────────────────────────────────────────────────────────────────────
⎪                             2⋅\lVert {\Delta} \rVert                        
⎪                                                                             
⎩                                                                             

                                                                              
                                                                              
              ⎛      \Delta₁⋅μ                 μ⋅u_{t1}      ⎞                
a₁)   \Delta₁⋅⎜────────────────────── - ───────────

In [158]:
A11_raw = flatten_piecewise(A_[1, 1]).args[0][0]
make_readable(A11_raw)

                                                                              
                                                                              
          ⎛        \Delta₁⋅μ                 μ⋅u_{t1}      ⎞                  
  \Delta₁⋅⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambda
          ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                  
- ────────────────────────────────────────────────────────────────────────────
                             2⋅\lVert {\Delta} \rVert                         

                                                                              
                                                                              
             ⎛      \Delta₁⋅μ                 μ⋅u_{t1}      ⎞                 
₁)   \Delta₁⋅⎜────────────────────── - ─────────────────────⎟⋅Heaviside(\lambd
             ⎝\lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                 
── + ──────────────────────────────────────────────

In [102]:
#mlimit(A_[1, 1].subs(ut1, t).subs(ut2, t), t, 0)
A11_ut0=nsimp(mlimit(piecewise_fold(A_[1, 1]).args[0][0].subs(ut1, t).subs(ut2, t), t, 0))
A11_ut0rt0=simplify(nsimp(mlimit(piecewise_fold(A_[1, 1]).args[0][0].subs(ut1, t).subs(ut2, t).subs(rt1, t).subs(rt2, 0), t, 0)))
simplify(make_readable(A11_ut0rt0))


         2                                     
μ⋅(μ - 1) ⋅Heaviside(\lambda_1__{\prime\prime})
───────────────────────────────────────────────
                    2                          
                 2⋅μ  - 2⋅μ + 1                

In [132]:
A11_ut0p = simplify(factor_try(A11_ut0.subs('rt1',rt1).subs('rt2',rt2), [rt1, rt1, rt2, rt2, sqrt(2), 2, sqrt(rt1**2+rt2**2)]))
make_readable(A11_ut0p)

  ⎛       ⎛       ⎛                                                           
μ⋅⎝r_{t1}⋅⎝r_{t1}⋅⎝2⋅\lVert {r_{t}} \rVert⋅(Heaviside(\lambda_1__{\prime}) + H
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                   ___                                        
eaviside(\lambda_2__{\prime})) + ╲╱ 2 ⋅r_{t1}⋅(-Heaviside(\lambda_1__{\prime})
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                  ⎞     ___       2                           
 + Heaviside(\lambda_2__{\prime}))⎠ + ╲╱ 2 ⋅r_{t2} ⋅(-Heaviside(\lambda_1__{\p
──────────────────────────────────────────────────

In [168]:
from sympy import Or
A[1, 1] = natfix(A11_raw)

In [169]:
# ||delta|| -> 0 && ||ut|| -> 0 ==> ||rt|| > 0
make_readable(A[1, 1])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎨                                                   

In [170]:
with open('codegen/tex/A11_fnat.tex', 'w') as f:
     f.write(latex(make_readable(A[1, 1])))

## A[1, 2]

In [171]:
make_readable(A_[1, 2])

⎧                                                                             
⎪                                                                            \
⎪\Delta₁⋅\Delta₂⋅μ⋅Max(0, \lambda_1)   \Delta₁⋅\Delta₂⋅μ⋅Max(0, \lambda_2)    
⎪─────────────────────────────────── - ─────────────────────────────────── - ─
⎨                             3                                     3         
⎪     2⋅\lVert {\Delta} \rVert              2⋅\lVert {\Delta} \rVert          
⎪                                                                             
⎪                                                                             
⎩                                                                             

       ⎛        \Delta₂⋅μ                 μ⋅u_{t2}      ⎞                     
Delta₁⋅⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambda₁) 
       ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                     
───────────────────────────────────────────────────

In [173]:
A12_raw=flatten_piecewise(A_[1, 2]).args[0][0]

In [175]:
A[1, 2] = natfix(A12_raw)
make_readable(A[1, 2])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎨                                                                             
⎪                                                   

In [176]:
with open('codegen/tex/A12_fnat.tex', 'w') as f:
     f.write(latex(make_readable(A[1, 2])))

## A[2, 0]

In [177]:
make_readable(A_[2, 0])

⎧\Delta₂⋅Heaviside(\lambda₁)   \Delta₂⋅Heaviside(\lambda₂)                    
⎪─────────────────────────── - ───────────────────────────  for \lVert {\Delta
⎪  2⋅\lVert {\Delta} \rVert      2⋅\lVert {\Delta} \rVert                     
⎨                                                                             
⎪         Heaviside(\lambda₁)   Heaviside(\lambda₂)                           
⎪       - ─────────────────── + ───────────────────         for \lVert {\Delta
⎩                  2                     2                                    

                   
} \rVert > \epsilon
                   
                   
                   
} \rVert ≤ \epsilon
                   

## A[2, 1]

In [178]:
make_readable(A_[2, 1])

⎧                                                                             
⎪                                                                            \
⎪\Delta₁⋅\Delta₂⋅μ⋅Max(0, \lambda_1)   \Delta₁⋅\Delta₂⋅μ⋅Max(0, \lambda_2)    
⎪─────────────────────────────────── - ─────────────────────────────────── - ─
⎪                             3                                     3         
⎪     2⋅\lVert {\Delta} \rVert              2⋅\lVert {\Delta} \rVert          
⎨                                                                             
⎪                                              ⎛        \Delta₁⋅μ             
⎪                                              ⎜- ────────────────────── - ───
⎪                                              ⎝  \lVert {\Delta} \rVert   \lV
⎪                                              ───────────────────────────────
⎪                                                                             
⎩                                                   

In [181]:
A21_raw = flatten_piecewise(A_[2, 1]).args[0][0]

A[2, 1] = natfix(A21_raw)

In [182]:
make_readable(A[2, 1])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎨                                                                             
⎪                                                   

In [0]:
N = sqrt(ut1**2+ut2**2)
mlimit(N.diff(ut2).subs(ut2, 381*t).subs(ut1, 2*t), t, 0)**2 + mlimit(N.diff(ut1).subs(ut2, 381*t).subs(ut1, 2*t), t, 0)**2

In [0]:
(sqrt(5)/5)**2 + (2*sqrt(5)/5)**2

In [183]:
with open('codegen/tex/A21_fnat.tex', 'w') as f:
     f.write(latex(make_readable(A[2, 1])))

## A[2, 2]

In [184]:
make_readable(A_[2, 2])

⎧                                                                             
⎪                                                                             
⎪          ⎛        \Delta₂⋅μ                 μ⋅u_{t2}      ⎞                 
⎪  \Delta₂⋅⎜- ────────────────────── - ─────────────────────⎟⋅Heaviside(\lambd
⎪          ⎝  \lVert {\Delta} \rVert   \lVert {u_{t}} \rVert⎠                 
⎪- ───────────────────────────────────────────────────────────────────────────
⎨                             2⋅\lVert {\Delta} \rVert                        
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎩                                                   

In [190]:
A22_raw = flatten_piecewise(A_[2, 2]).args[0][0]

In [192]:
A[2, 2] = natfix(A22_raw)
make_readable(A[2, 2])

⎧                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎪                                                                             
⎨                                                   

In [0]:
with open('codegen/tex/A22_fnat.tex', 'w') as f:
     f.write(latex(make_readable(A[2, 2])))